In [22]:
import numpy as np;
import pandas as pd;
import tensorflow as tf;
import spacy;

In [23]:
def read_file(filepath):
    with open(filepath) as f:
        str_text=f.read()
        return str_text

In [24]:
nlp=spacy.load("en_core_web_lg")

In [25]:
nlp.max_length=5000000
def seperate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n \n\n\n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [26]:
d=read_file("Book1.txt")
tokens=seperate_punc(d)

In [27]:
train_len=25+1
text_sequences=[]
for i in range(train_len,len(tokens)):
    seq=tokens[i-train_len:i]
    text_sequences.append(seq)

In [28]:
' '.join(text_sequences[10])

'four privet drive were proud to say that they were perfectly normal thank you very much they were the last people you ’d expect to be'

In [29]:
from keras.preprocessing.text import Tokenizer;
tokenizer=Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences=tokenizer.texts_to_sequences(text_sequences)

In [30]:
np.array((sequences[12])) #words have been represented as ID

array([ 530,   44, 1331,    5,  168,   22,   21,   44, 1680, 1018, 1097,
         12,   84,  165,   21,   44,    1,  154,  134,   12,   52,  743,
          5,   38, 1479,   14])

In [31]:
for i in sequences[1]:
    print(f"{i}:{tokenizer.index_word[i]}")

152:boy
72:who
1098:lived
158:mr.
4:and
264:mrs.
222:dursley
9:of
613:number
330:four
646:privet
530:drive
44:were
1331:proud
5:to
168:say
22:that
21:they
44:were
1680:perfectly
1018:normal
1097:thank
12:you
84:very
165:much
21:they


In [32]:
tokenizer.word_counts

OrderedDict([('the', 103303),
             ('boy', 2186),
             ('who', 5151),
             ('lived', 212),
             ('mr.', 2059),
             ('and', 58979),
             ('mrs.', 1177),
             ('dursley', 1464),
             ('of', 32890),
             ('number', 426),
             ('four', 895),
             ('privet', 402),
             ('drive', 507),
             ('were', 8614),
             ('proud', 171),
             ('to', 48345),
             ('say', 1941),
             ('that', 17880),
             ('they', 17933),
             ('perfectly', 125),
             ('normal', 256),
             ('thank', 231),
             ('you', 26674),
             ('very', 4471),
             ('much', 1976),
             ('last', 2158),
             ('people', 2496),
             ('’d', 6864),
             ('expect', 338),
             ('be', 9516),
             ('involved', 130),
             ('in', 25142),
             ('anything', 1872),
             ('strange', 598),
 

In [33]:
vocab_size=len(tokenizer.word_counts)
print(vocab_size)

6170


In [34]:
type(sequences)

list

In [35]:
sequences=np.array(sequences)

In [36]:
sequences

array([[   1,  152,   72, ...,   12,   84,  165],
       [ 152,   72, 1098, ...,   84,  165,   21],
       [  72, 1098,  158, ...,  165,   21,   44],
       ...,
       [ 421,  254,   24, ...,    4,    1,   40],
       [ 254,   24,  593, ...,    1,   40,   30],
       [  24,  593,   16, ...,   40,   30,   43]])

In [37]:
from tensorflow.keras.utils import to_categorical;
X=sequences[:,:-1]
Y=sequences[:,-1]

In [38]:
Y=to_categorical(Y,num_classes=(vocab_size+1))# extra padding is needed for keras to input 0

In [39]:
seq_len=X.shape[1]

In [40]:
tf.random.set_seed(42)
model_1=tf.keras.Sequential()
model_1.add(tf.keras.layers.Embedding(input_dim=vocab_size+1,output_dim=seq_len,input_length=seq_len)) 
model_1.add(tf.keras.layers.LSTM(64,return_sequences=True))
model_1.add(tf.keras.layers.LSTM(64))
model_1.add(tf.keras.layers.Dense(128,activation="relu"))
model_1.add(tf.keras.layers.Dense(vocab_size+1,activation="softmax"))
    

model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])

In [41]:
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            154275    
_________________________________________________________________
lstm_2 (LSTM)                (None, 25, 64)            23040     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 6171)              796059    
Total params: 1,014,718
Trainable params: 1,014,718
Non-trainable params: 0
_________________________________________________________________


In [42]:
model_1.fit(X,Y,epochs=800,batch_size=128,verbose=1)

Epoch 1/800
706/706 [==============================] - 11s 12ms/step - loss: 6.4891 - accuracy: 0.0437
Epoch 2/800
706/706 [==============================] - 9s 12ms/step - loss: 6.1064 - accuracy: 0.0532
Epoch 3/800
706/706 [==============================] - 7s 10ms/step - loss: 5.8501 - accuracy: 0.0721
Epoch 4/800
706/706 [==============================] - 7s 10ms/step - loss: 5.6671 - accuracy: 0.0895
Epoch 5/800
706/706 [==============================] - 7s 9ms/step - loss: 5.4690 - accuracy: 0.1176
Epoch 6/800
706/706 [==============================] - 7s 10ms/step - loss: 5.2959 - accuracy: 0.1357
Epoch 7/800
706/706 [==============================] - 8s 12ms/step - loss: 5.1546 - accuracy: 0.1462
Epoch 8/800
706/706 [==============================] - 9s 13ms/step - loss: 5.0237 - accuracy: 0.1564
Epoch 9/800
706/706 [==============================] - 9s 12ms/step - loss: 4.8939 - accuracy: 0.1646
Epoch 10/800
706/706 [==============================] - 9s 12ms/step - loss: 4.780

In [43]:
import pickle;
model_1.save("LSTM_Model.h5")
pickle.dump(tokenizer,open("harry_tokenizer","wb"))

In [44]:
from keras.preprocessing.sequence import pad_sequences
def generate_text(model,tokenizer,seq_len,seed_text,num_gen_words):
    output_text=[]
    input_text=seed_text
    for i in range(num_gen_words):
        encoded_text=tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded=pad_sequences([encoded_text],maxlen=seq_len,truncating="pre")
        pred_word_ind=model.predict(pad_encoded,verbose=0)[0]
        pred_word_ind=np.argmax(pred_word_ind)
        pred_word=tokenizer.index_word[pred_word_ind]
        input_text +=' ' +pred_word
        output_text.append(pred_word)
    return ' '.join(output_text)

In [45]:
import random;
random.seed(101)
random_pick=random.randint(0,len(text_sequences))
random_seed_text=text_sequences[random_pick]
seed_text=' '.join(random_seed_text)

In [46]:
generate_text(model_1,tokenizer=tokenizer,seq_len=seq_len,seed_text=seed_text,num_gen_words=50)

'back his sheets he found his invisibility hand hung anything to see something but the hut they and ron thought that ’s head and tried to remember anything the steps in a boat he could n’t often touch a very last indeed percy taller than saying his broom around “'